In [1]:
from nustar_pysolar import planning
from nustar_pysolar.planning import _parse_timestamp as parse_timestamp
import astropy.units as u
from astropy.time import Time
from astropy.coordinates import get_body, solar_system_ephemeris, get_body_barycentric, SkyCoord
from datetime import datetime

In [2]:
fname = planning.download_occultation_times(outdir='../data/')
print(fname)

../data/NUSTAR.2017_121.SHADOW_ANALYSIS.txt


In [3]:
tstart = '2017-05-19T01:43:00'
tend = '2017-05-19T11:43:00'

tstamp = '2017:137:09:15:00'
tstart = datetime.strptime(tstamp, '%Y:%j:%H:%M:%S')                
tstamp = '2017:140:13:05:00'
tend = datetime.strptime(tstamp, '%Y:%j:%H:%M:%S')                

orbits = planning.sunlight_periods(fname, tstart, tend)
solar_system_ephemeris.set('jpl')



<ScienceState solar_system_ephemeris: 'jpl'>

In [4]:
from skyfield.api import load
ts = load.timescale()
planets = load('jup310.bsp')


# Use the astropy interface to get the location of Juptier as the time that you want to use.

In [14]:
dt = 0.

# Using JPL Horizons web interface at 2017-05-19T01:34:40
horizon_ephem = SkyCoord(*[193.1535, -4.01689]*u.deg)

f = open('jupiter_20170517to20170520_pointing.txt', 'w')
for orbit in orbits:
    tstart = orbit[0]
    tend = orbit[1]
    print()
#    print('Orbit duration: ', tstart.isoformat(), tend.isoformat())
    on_time = (tend - tstart).total_seconds()
    
    point_time = tstart + 0.5*(tend - tstart)
#    print('Time used for ephemeris: ', point_time.isoformat())
    
    astro_time = Time(point_time)
    solar_system_ephemeris.set('jpl')


    jupiter = get_body('Jupiter', astro_time)
    
    jplephem = SkyCoord(jupiter.ra.deg*u.deg, jupiter.dec.deg*u.deg)
    
    # Switch to the built in ephemris
    solar_system_ephemeris.set('builtin')
    jupiter = get_body('Jupiter', astro_time)
    
    builtin_ephem = SkyCoord(jupiter.ra.deg*u.deg, jupiter.dec.deg*u.deg)
    
    t = ts.from_astropy(astro_time)
    jupiter, earth = planets['jupiter'], planets['earth']
    astrometric = earth.at(t).observe(jupiter)
    ra, dec, distance = astrometric.radec()
    radeg = ra.to(u.deg)
    decdeg = dec.to(u.deg)
    skyfield_ephem = SkyCoord(radeg, decdeg)
    
    dt += on_time
#     print()
#     print('Horizons offset to jplephem: ', horizon_ephem.separation(jplephem))
#     print()
#     print('Horizons offset to "built in" ephemeris: ', horizon_ephem.separation(builtin_ephem))
#     print()
#     print('Horizons offset to Skyfield ephemeris: ', horizon_ephem.separation(skyfield_ephem))

    print()
#    print('Orbit start:  RA  Dec '.format(radeg, decdeg))

    print(tstart.isoformat()+' {}  {}'.format(radeg.value, decdeg.value))
    f.write(tstart.isoformat()+' {}  {}'.format(radeg.value, decdeg.value)+'\n')
    
f.close()
#    break
    
print(dt)



2017-05-17T08:46:20 193.25770369661515  -4.053693804129604


2017-05-17T10:23:00 193.2534024955027  -4.052163126939793


2017-05-17T11:59:40 193.24911237063824  -4.050637312138953


2017-05-17T13:36:20 193.244833315825  -4.049116356382934


2017-05-17T15:13:00 193.24056165219164  -4.0475989525196185


2017-05-17T16:49:40 193.23630473682803  -4.046087713741976


2017-05-17T18:26:30 193.23205523224345  -4.044580035456602


2017-05-17T20:03:10 193.2278204651818  -4.0430785179108915


2017-05-17T21:39:50 193.22359678142345  -4.041581865504474


2017-05-17T23:16:30 193.21938419618672  -4.0400900853296


2017-05-18T00:53:10 193.215182729002  -4.038603186339905


2017-05-18T02:29:50 193.21099240304756  -4.037121179004338


2017-05-18T04:06:30 193.20680964675793  -4.0356428037695435


2017-05-18T05:43:10 193.20264169391663  -4.034170619741318


2017-05-18T07:19:50 193.19848496653748  -4.03270336410748


2017-05-18T08:56:30 193.19433949433235  -4.031241049335848


2017-05-18T10:33:10 193.1902